In [ ]:
import numpy as np
from linear_svm import *
from softmax import *
from sklearn import datasets,cross_validation
import torch
#from torch.autograd import grad

class LinearClassifier(object):

  def __init__(self):
    self.W = None

  def tracin(self,grad_z_train,batch_size,W,iters,reg):
    data=datasets.load_iris()
    X_train = data.data
    y_train = data.target
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_train, y_train, test_size=0.25,
                                                                         random_state=0, stratify=y_train)
    num_training = 100
    num_validation = 12
    num_test = 38
    num_dev = 50
    mask = range(num_training, num_training + num_validation)
    X_tracin = X_train[mask]
    y_tracin = y_train[mask]
    num_tracin, dim = X_tracin.shape
    num_classes = np.max(y_tracin) + 1

    if self.W is None:
      # lazily initialize W
      self.W = 0.001 * np.random.randn(dim, num_classes)

    # Run stochastic gradient descent to optimize W
    loss_history = []
    for it in range(num_tracin):
      X_batch = None
      y_batch = None

      batch_idx = np.random.choice(num_tracin, batch_size, replace=True)  # 这里是随机选一个，考虑锚点的话应该改成顺序选择
      print("this point is Point_{}".format(batch_idx))
      X_batch = X_tracin[batch_idx]
      y_batch = y_tracin[batch_idx]

      # evaluate loss and gradient
      loss, grad = self.loss(X_batch, y_batch, reg)
      grad_z_test = grad#self.get_grad(grad, batch_idx)
      score=tracin_get(grad_z_train,grad_z_test)
      ###这里插入tracin，但是要考虑到循环问题
      loss_history.append(loss)
  def train(self, X, y, learning_rate=1e-3, reg=1e-5, num_iters=100,
            batch_size=1, verbose=False):
    """
    Train this linear classifier using stochastic gradient descent.

    Inputs:
    - X: A numpy array of shape (N, D) containing training data; there are N
      training samples each of dimension D.
    - y: A numpy array of shape (N,) containing training labels; y[i] = c
      means that X[i] has label 0 <= c < C for C classes.
    - learning_rate: (float) learning rate for optimization.
    - reg: (float) regularization strength.
    - num_iters: (integer) number of steps to take when optimizing
    - batch_size: (integer) number of training examples to use at each step.
    - verbose: (boolean) If true, print progress during optimization.

    Outputs:
    A list containing the value of the loss function at each training iteration.
    """
    num_train, dim = X.shape
    num_classes = np.max(y) + 1 # assume y takes values 0...K-1 where K is number of classes
    if self.W is None:
      # lazily initialize W
      self.W = 0.001 * np.random.randn(dim, num_classes)

    # Run stochastic gradient descent to optimize W
    loss_history = []
    for it in range(num_iters):
      X_batch = None
      y_batch = None

      batch_idx = np.random.choice(num_train, batch_size, replace = True)#这里是随机选一个，考虑锚点的话应该改成顺序选择
      print("this point is Point_{}".format(batch_idx))
      X_batch =  X[batch_idx]
      y_batch = y[batch_idx]

      # evaluate loss and gradient
      loss, grad = self.loss(X_batch, y_batch, reg)
      #grad_z_train=grad#self.get_grad(grad,batch_idx)
      #tracin_score=self.tracin(grad_z_train,batch_size,100,reg)
      #print("Tracin score for SVM on sample {} is {}".format(batch_idx,tracin_score))
      ###这里插入tracin，但是要考虑到循环问题
      loss_history.append(loss)

      # perform parameter update

      self.W += - learning_rate * grad


      if verbose and it % 100 == 0:
        print('iteration %d / %d: loss %f' % (it, num_iters, loss))

    return loss_history

  def predict(self, X):
    """
    Use the trained weights of this linear classifier to predict labels for
    data points.

    Inputs:
    - X: D x N array of training data. Each column is a D-dimensional point.

    Returns:
    - y_pred: Predicted labels for the data in X. y_pred is a 1-dimensional
      array of length N, and each element is an integer giving the predicted
      class.
    """
    y_pred = np.zeros(X.shape[1])
    ###########################################################################
    # TODO:                                                                   #
    # Implement this method. Store the predicted labels in y_pred.            #
    ###########################################################################
    scores = X.dot(self.W)
    y_pred = np.argmax(scores, axis = 1)
    #pass
    ###########################################################################
    #                           END OF YOUR CODE                              #
    ###########################################################################
    return y_pred
  
  def loss(self, X_batch, y_batch, reg):
    """
    Compute the loss function and its derivative. 
    Subclasses will override this.

    Inputs:
    - X_batch: A numpy array of shape (N, D) containing a minibatch of N
      data points; each point has dimension D.
    - y_batch: A numpy array of shape (N,) containing labels for the minibatch.
    - reg: (float) regularization strength.

    Returns: A tuple containing:
    - loss as a single float
    - gradient with respect to self.W; an array of the same shape as W
    """
    pass


class LinearSVM(LinearClassifier):
  """ A subclass that uses the Multiclass SVM loss function """

  def loss(self, X_batch, y_batch, reg):
    return svm_loss_vectorized(self.W, X_batch, y_batch, reg)


class Softmax(LinearClassifier):
  """ A subclass that uses the Softmax + Cross-entropy loss function """

  def loss(self, X_batch, y_batch, reg):
    return softmax_loss_vectorized(self.W, X_batch, y_batch, reg)

